In [1]:
import bagpy
import pandas as pd
import sys
import numpy as np
import os
from scipy.spatial.transform import Rotation as R

conversion = 0.18
file_path = '/home/shc/Desktop/data/0220/phacon_pose.bag'
b = bagpy.bagreader(file_path)
b.topic_table

[INFO]  Data folder /home/shc/Desktop/data/0220/phacon_pose already exists. Not creating.


,Topics,Types,Message Count,Frequency
0,/fwd_limage/compressed,sensor_msgs/CompressedImage,39,30.095388
1,/fwd_pose_camhand,geometry_msgs/PoseStamped,39,30.031175
2,/fwd_pose_pan,geometry_msgs/PoseStamped,40,29.713330
3,/fwd_rimage/compressed,sensor_msgs/CompressedImage,39,30.071761


In [2]:
pose_cam_data = b.message_by_topic('/fwd_pose_camhand')
pose_pan_data = b.message_by_topic('/fwd_pose_pan')

df_pan = pd.read_csv(pose_pan_data)[:8449]
df_cam_hand = pd.read_csv(pose_cam_data)[:8449]

In [3]:
os.chdir('/home/shc/RoboMaster/util')
from Solver import solver
from dataLoader import dataLoader
sol = solver()
ld = dataLoader()

phacon2pan = np.load('/home/shc/RoboMaster/params/phacon2pan_1028.npy')
# pan2phacon = np.load('/home/shc/RoboMaster/params/phacon2pan.npy')

X = np.load("/home/shc/Desktop/data/0205/handeye_1/hand_eye_X.npy")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
def getRealPose(idx, pd_data):
    df = pd_data
    pose_x = df['pose.position.x'][idx]*1000
    pose_y = df['pose.position.y'][idx]*1000
    pose_z = df['pose.position.z'][idx]*1000
    orin_x = df['pose.orientation.x'][idx]
    orin_y = df['pose.orientation.y'][idx]
    orin_z = df['pose.orientation.z'][idx]
    orin_w = df['pose.orientation.w'][idx]
    pose = np.array([pose_x, pose_y, pose_z, orin_x, orin_y, orin_z, orin_w])
    # print(real_pose, '\n', pose)
    return pose

In [5]:
num_frames = len(df_cam_hand)
pose_pan_data = np.zeros([num_frames, 4, 4])
pose_camhand_data = np.zeros([num_frames, 4, 4])
for i in range(num_frames):
    quaternion_pan = getRealPose(i, df_pan)
    quaternion_camhand = getRealPose(i, df_cam_hand)
    _, op2pan = sol.seven2trans(quaternion_pan)
    _, op2camhand = sol.seven2trans(quaternion_camhand)
    
    pan2phacon = sol.invTransformation(phacon2pan)
    op2phacon = op2pan@pan2phacon
    op2cam = op2camhand@X
     ## F_cv_ambf, cam2ambf
    extrinsic = np.array([[0, 1, 0, 0], [0, 0, -1, 0],
                          [-1, 0, 0, 0], [0, 0, 0, 1]])
    # op2cam = op2cam@extrinsic
    cam2phacon = sol.invTransformation(op2cam)@op2phacon
print(cam2phacon)
# r = R.from_matrix(cam2phacon[:3,:3])
# r.as_euler('xyz', degrees=True)


[[-1.56847866e-01  2.11025709e-02  9.87397300e-01 -1.98847206e+01]
 [-1.39823488e-01 -9.90175889e-01 -1.04897874e-03  7.39939098e+01]
 [ 9.77674864e-01 -1.38225864e-01  1.58257610e-01  1.15987407e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [6]:
op2pan

array([[ 8.94721690e-01, -7.34614660e-02,  4.40541156e-01,
         6.96227264e+01],
       [ 5.54019471e-02,  9.97017073e-01,  5.37362101e-02,
         1.44012131e+02],
       [-4.43174594e-01, -2.36721149e-02,  8.96122709e-01,
         1.03624622e+03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [7]:
pan2phacon

array([[ 3.70575564e-01, -6.35414444e-03,  9.28780586e-01,
        -5.61059078e+01],
       [ 2.22972990e-01, -9.70125472e-01, -9.56013298e-02,
        -9.56255793e+01],
       [ 9.01641169e-01,  2.42520501e-01, -3.58087992e-01,
         3.64185446e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [8]:
ambf2phacon = sol.invTransformation(extrinsic)@cam2phacon

In [9]:
t = ambf2phacon[:3, 3]/conversion/1000
r = R.from_matrix(ambf2phacon[:3,:3])
Rot = r.as_euler('xyz')
# Rot = Rot@Rot_offset
print('position: {{x: {0}, y: {1}, z: {2}}}'.format(t[0],t[1],t[2]))
print('orientation: {{r: {0}, p: {1}, y: {2}}}'.format(Rot[0], Rot[1], Rot[2]))

position: {x: -0.6443744848333767, y: -0.11047066982653998, z: -0.41107727643651104}
orientation: {r: 1.5697369409205635, p: -0.14028314890895355, y: -2.9825186636935506}


Fix the phacon to the AMBF origin

In [10]:
phacon2op = sol.invTransformation(op2pan@pan2phacon)
phacon2op

array([[ 7.12392115e-01,  2.91289265e-01,  6.38473209e-01,
        -7.14333562e+02],
       [ 1.72421910e-01, -9.54551558e-01,  2.43109046e-01,
        -2.20466848e+02],
       [ 6.80270652e-01, -6.31021967e-02, -7.30239655e-01,
         7.62705513e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])